find . -name "__pycache__" -type d -exec rm -rf {} +

python apollo_service/test.py
ngrok http 5000

# deployment steps:
0) 
1) make sure that Dockerfile, requirements.txt and main.py is in repo
2) export vars to ~/.zshrc
!!!full paths to the credentials.json and token.json this is for local run
change here GOOGLE_CLOUD_PROJECT, GOOGLE_OAUTH_CLIENT_FILE, GOOGLE_OAUTH_TOKEN_FILE
```
export GOOGLE_GENAI_USE_VERTEXAI=True
export GOOGLE_CLOUD_PROJECT=personalplanner-476218       
export GOOGLE_CLOUD_LOCATION=us-central1
export GOOGLE_OAUTH_CLIENT_FILE="/Users/gman/Documents/hackathon/personalPlanner/.creds/credentials.json"  
export GOOGLE_OAUTH_TOKEN_FILE="/Users/gman/Documents/hackathon/personalPlanner/.creds/token.json"
export MODEL=gemini-2.5-flash
```
3) ```source ~/.zshrc```
NOTE to run the current verison of setup run with python main.py. the above exports are for local run only, not cloudrun!!!

4) deploy containers ( change project name in 2 places here)

```
gcloud run deploy project-planner-service \
  --source . \
  --region us-central1 \
  --project personalplanner-476218 \
  --allow-unauthenticated \
  --memory=1Gi \
  --set-env-vars=GOOGLE_GENAI_USE_VERTEXAI=True,GOOGLE_CLOUD_PROJECT=personalplanner-476218,GOOGLE_CLOUD_LOCATION=us-central1,MODEL=gemini-2.5-flash,GOOGLE_OAUTH_CLIENT_FILE=/app/.creds/credentials.json,GOOGLE_OAUTH_TOKEN_FILE=/app/.creds/token.json
  
  
  # maybe this:

gcloud builds submit \
  --tag gcr.io/personalplanner-476218/project-planner-service .

  # then 

gcloud run deploy project-planner-service \
  --image gcr.io/personalplanner-476218/project-planner-service \
  --region us-central1 \
  --project personalplanner-476218 \
  --allow-unauthenticated \
  --memory=1Gi \
  --set-env-vars=GOOGLE_GENAI_USE_VERTEXAI=True,GOOGLE_CLOUD_PROJECT=personalplanner-476218,GOOGLE_CLOUD_LOCATION=us-central1,MODEL=gemini-2.5-flash,GOOGLE_OAUTH_CLIENT_FILE=/app/.creds/credentials.json,GOOGLE_OAUTH_TOKEN_FILE=/app/.creds/token.json

## then his orchestrator with this:
curl https://project-planner-service-98380938461.us-central1.run.app/healthz


```


5) run this ( Change project name )
```
gcloud run services describe project-planner-service \
  --region=us-central1 \
  --project=personalplanner-476218 \
  --format='value(spec.template.spec.serviceAccountName)'
```


it returns something like xxxxxxx-compute@developer.gserviceaccount.com
then run what's below with that thingy as your SA_EMAIL
```
SA_EMAIL="98380938461-compute@developer.gserviceaccount.com"      

gcloud projects add-iam-policy-binding personalplanner-476218 \
  --member="serviceAccount:${SA_EMAIL}" \
  --role="roles/aiplatform.user"

```
6) run this if session changed
```
gcloud run services update-traffic project-planner-service \
  --region us-central1 \
  --to-latest
```
7) idk anymore force cloud run to a single instance
```
gcloud run services update project-planner-service \
  --region us-central1 \
  --max-instances=1 \
  --min-instances=1

```

# Paste your service URL here 

In [63]:
import requests
import json
# Service [project-planner-service] revision [project-planner-service-00002-fhq] has been deployed and is serving 100 percent of traffic.
# Service URL: https://project-planner-service-98380938461.us-central1.run.app
SERVICE_URL = "https://project-planner-service-98380938461.us-central1.run.app"

# Helpers

In [64]:
APP_NAME = "orchestrator"  # from /list-apps
USER_ID = "grant"          # anything you like
SESSION_ID = "sess1"       # anything you like
def pretty_print(obj):
    print(json.dumps(obj, indent=2, ensure_ascii=False))


# 2) List available apps
def list_apps():
    url = f"{SERVICE_URL}/list-apps"
    resp = requests.get(url)
    resp.raise_for_status()
    apps = resp.json()
    print("Available apps:")
    pretty_print(apps)
    return apps


# 3) Create or update a session

def create_session(initial_state=None):
    if initial_state is None:
        initial_state = {}

    url = f"{SERVICE_URL}/apps/{APP_NAME}/users/{USER_ID}/sessions/{SESSION_ID}"
    resp = requests.post(url, json=initial_state)

    print("Status:", resp.status_code)
    print("Headers:", resp.headers)
    print("Body:\n", resp.text)

    # Only raise if it's 4xx/5xx *after* we print
    resp.raise_for_status()

    try:
        session = resp.json()
        print("\nParsed session JSON:")
        pretty_print(session)
        return session
    except Exception as e:
        print("\nJSON parse error:", e)
        return None



# 4) Get session (optional, for debugging)
def get_session():
    url = f"{SERVICE_URL}/apps/{APP_NAME}/users/{USER_ID}/sessions/{SESSION_ID}"
    resp = requests.get(url)
    resp.raise_for_status()
    session = resp.json()
    print("Current session:")
    pretty_print(session)
    return session


# 5) Run the orchestrator with a user message
def run_orchestrator(message_text: str):
    url = f"{SERVICE_URL}/run"
    payload = {
        "app_name": APP_NAME,
        "user_id": USER_ID,
        "session_id": SESSION_ID,
        "new_message": {
            "role": "user",
            "parts": [
                {"text": message_text}
            ]
        }
    }
    resp = requests.post(url, json=payload)
    print("Status:", resp.status_code)
    print("Body:\n", resp.text)
    resp.raise_for_status()
    events = resp.json()
    print("Events from orchestrator:")
    pretty_print(events)
    return events


# 6) Convenience helper: full flow in one call
def send_to_orchestrator(message_text: str, init_if_needed: bool = True):
    """
    - Optionally creates/updates a session first
    - Sends a message
    - Returns events
    """
    if init_if_needed:
        # you can attach any initial session state here if you want
        create_session(initial_state={})

    return run_orchestrator(message_text)


# 7) Delete session (if you want to reset state)
def delete_session():
    url = f"{SERVICE_URL}/apps/{APP_NAME}/users/{USER_ID}/sessions/{SESSION_ID}"
    resp = requests.delete(url)
    if resp.status_code not in (200, 204):
        print("Delete returned:", resp.status_code, resp.text)
    else:
        print("Session deleted.")



In [65]:
apps = list_apps()

Available apps:
[
  "apollo_service",
  "calendar_service",
  "call_service",
  "gmail_service",
  "google_docs_service",
  "google_drive_service",
  "google_search_service",
  "google_sheets_service",
  "jobs_service",
  "matching_service",
  "orchestrator",
  "personalplanner.egg-info",
  "resume_customization",
  "resume_customization_service",
  "services",
  "ui",
  "utils"
]


In [66]:
create_session(initial_state={"note": "session created from Jupyter"})


Status: 200
Headers: {'content-type': 'application/json', 'x-cloud-trace-context': 'e337183e7c5bbe66cedac7b3fbe57e12', 'date': 'Mon, 10 Nov 2025 17:36:19 GMT', 'server': 'Google Frontend', 'Content-Length': '146', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'}
Body:
 {"id":"sess1","appName":"orchestrator","userId":"grant","state":{"note":"session created from Jupyter"},"events":[],"lastUpdateTime":1762796179.0}

Parsed session JSON:
{
  "id": "sess1",
  "appName": "orchestrator",
  "userId": "grant",
  "state": {
    "note": "session created from Jupyter"
  },
  "events": [],
  "lastUpdateTime": 1762796179.0
}


{'id': 'sess1',
 'appName': 'orchestrator',
 'userId': 'grant',
 'state': {'note': 'session created from Jupyter'},
 'events': [],
 'lastUpdateTime': 1762796179.0}

In [40]:
events = run_orchestrator(
    """Task: Customize my resume for this job.

Job title: Machine Learning Engineer  
Company: Stripe  
Job_row_id_in_sheet: 42  (if applicable)

Target skills for this job:
Machine Learning, Deep Learning, PyTorch, JAX, C++, Communication

What is missing:
The resume does not explicitly mention experience with JAX. While communication is implied
through project work and team collaboration, it is not explicitly highlighted as a distinct
skill, which could be beneficial for roles emphasizing this aspect.

Please update the LaTeX resume, rebuild the PDF, upload it to the configured Drive folder,
and return ONLY a single JSON object with:
status, job_title, company, drive_file_id, summary_of_changes.
""")


Status: 200
Body:
 [{"modelVersion":"gemini-2.5-flash","content":{"parts":[{"functionCall":{"id":"adk-6ed007a3-0e9e-41b9-b40b-0753f5f9b5df","args":{"agent_name":"resume_customization_agent"},"name":"transfer_to_agent"},"thoughtSignature":"CvoFAePx_15n3uRzfSCtpg5v0gagtQGg2LF2-jlQ1LTGU37NW_RpV-vzBXq9yZRju8S2AG4KH5uyQd6vBUbt_7FntyO3u6PMSwLVgeYSIWGcEv-5RstwR07HP688adDc92n4nCyzBGBVM54p1J2edRV0aKMK1-6epWygWQ5fPpYyJPdadij_Y4yhFCz123MjfkoRycP_UlDAqfD2TBBlrB5aWBFT8IgaouaL2RHEiSZ1U7Dk9v2VjXO9LgGUdFQAQ5N6zE2mEu4vGXGDlJhk8YJ7Pueqt-zsKQHl_afxYMZHnGLStHhsb-omnsA_YTqSF3qz_6yCnGwQlnz9RnMBww8HsA_v4u-Gna8IiDQDQtrS_-VwDZPTBk2dk-QRYQmrqv7n0NC11p6yQJgc9GJlZ_qDxBR0tGMkd5hhEwD7KlIlo9O9tBKr_qxjm6eTk_UVzURuhHHRKdVSGpKW385igWQsILZ5cHJpqG3Avtq1uSHTMnBWjmkLzwRPai2x3SLg3iUisvKa29ZXSzRm2QFNum_W2vkULKjsWJ-glIQE7Ocg5ofeCnBN7Da5WkvJVT6mFInHL403XilY3NndekiDjsJDflbHbFOn1RfZqNBUzr3_jUAQbvwWgMKbSakXJ5MzfTZkCObwgqC2bN_PBR8Z_irkmJMjXOrsXL6U_3dJ5RlaRqvoMFHTXZx-HiSzck9fou6mpVSiYmyjCzQrr8NKHNVVZOktw4kheYBuDJe0MtYj7IRQ3tZ5jYZqI

In [41]:
events

[{'modelVersion': 'gemini-2.5-flash',
  'content': {'parts': [{'functionCall': {'id': 'adk-6ed007a3-0e9e-41b9-b40b-0753f5f9b5df',
      'args': {'agent_name': 'resume_customization_agent'},
      'name': 'transfer_to_agent'},
     'thoughtSignature': 'CvoFAePx_15n3uRzfSCtpg5v0gagtQGg2LF2-jlQ1LTGU37NW_RpV-vzBXq9yZRju8S2AG4KH5uyQd6vBUbt_7FntyO3u6PMSwLVgeYSIWGcEv-5RstwR07HP688adDc92n4nCyzBGBVM54p1J2edRV0aKMK1-6epWygWQ5fPpYyJPdadij_Y4yhFCz123MjfkoRycP_UlDAqfD2TBBlrB5aWBFT8IgaouaL2RHEiSZ1U7Dk9v2VjXO9LgGUdFQAQ5N6zE2mEu4vGXGDlJhk8YJ7Pueqt-zsKQHl_afxYMZHnGLStHhsb-omnsA_YTqSF3qz_6yCnGwQlnz9RnMBww8HsA_v4u-Gna8IiDQDQtrS_-VwDZPTBk2dk-QRYQmrqv7n0NC11p6yQJgc9GJlZ_qDxBR0tGMkd5hhEwD7KlIlo9O9tBKr_qxjm6eTk_UVzURuhHHRKdVSGpKW385igWQsILZ5cHJpqG3Avtq1uSHTMnBWjmkLzwRPai2x3SLg3iUisvKa29ZXSzRm2QFNum_W2vkULKjsWJ-glIQE7Ocg5ofeCnBN7Da5WkvJVT6mFInHL403XilY3NndekiDjsJDflbHbFOn1RfZqNBUzr3_jUAQbvwWgMKbSakXJ5MzfTZkCObwgqC2bN_PBR8Z_irkmJMjXOrsXL6U_3dJ5RlaRqvoMFHTXZx-HiSzck9fou6mpVSiYmyjCzQrr8NKHNVVZOktw4kheYBuDJe0MtYj

In [42]:
import json

def extract_final_json(events):
    final_text = None

    for ev in events:
        for part in ev.get("content", {}).get("parts", []):
            txt = part.get("text")
            if txt and '"drive_file_id"' in txt:
                final_text = txt

    if final_text is None:
        raise RuntimeError("No JSON response with drive_file_id found")

    s = final_text.strip()
    # Strip ```json ... ``` if present
    if s.startswith("```"):
        # Remove leading ```json or ``` 
        if s.startswith("```json"):
            s = s[len("```json"):].strip()
        elif s.startswith("```"):
            s = s[3:].strip()
        # Remove trailing ```
        if s.endswith("```"):
            s = s[:-3].strip()

    payload = json.loads(s)
    return payload

payload = extract_final_json(events)
drive_file_id = payload["drive_file_id"]
summary_of_changes = payload["summary_of_changes"]

print("drive_file_id:", drive_file_id)
print("summary_of_changes:", summary_of_changes)


drive_file_id: 1J2HiMtMmhM98Y5hY-ZkXkZgSdFUI-cGx
summary_of_changes: Added JAX to ML/Distributed Training skills and 'Communication' under Professional Skills.


In [43]:
def extract_file_id_from_tool(events):
    for ev in events:
        for part in ev.get("content", {}).get("parts", []):
            fr = part.get("functionResponse")
            if fr and fr.get("name") == "upload_built_resume_to_drive":
                resp = fr.get("response", {})
                return resp.get("file_id")
    return None

file_id_from_tool = extract_file_id_from_tool(events)
print("file_id_from_tool:", file_id_from_tool)


file_id_from_tool: 1J2HiMtMmhM98Y5hY-ZkXkZgSdFUI-cGx


In [ ]:
run_orchestrator(
    """Tag jobs in the sheet that match 5 years experience in San Francisco, including remote jobs that qualify."""
)

# trash

In [10]:
from orchestrator_client import create_session, run_orchestrator, list_apps

list_apps()  # should show "orchestrator" in the list

create_session(initial_state={"note": "session created from REPL"})

run_orchestrator("can you look for software engineer jobs posted for the past 7 days")


Available apps:
[
  "apollo_service",
  "calendar_service",
  "gmail_service",
  "google_docs_service",
  "google_drive_service",
  "google_search_service",
  "google_sheets_service",
  "jobs_service",
  "matching_service",
  "orchestrator",
  "personalplanner.egg-info",
  "resume_customization",
  "services",
  "ui",
  "utils"
]

[create_session]
Status: 409
Body: {"detail":"Session already exists: sess1"} 
Session already exists; reusing it.

[run_orchestrator]
Status: 500
Body: Internal Server Error 


HTTPError: 500 Server Error: Internal Server Error for url: https://project-planner-service-98380938461.us-central1.run.app/run

In [ ]:



create_session(initial_state={"note": "session from REPL"})
run_orchestrator("can you look for software engineer jobs posted for the past 7 days")
